In [2]:
import pandas as pd

In [159]:
data = pd.read_csv("OnlineRetail.csv")
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [160]:
# dataset size
len(data)

541909

In [161]:
# number of transactions
len(data["InvoiceNo"].unique())

25900

In [162]:
# remove irrelevant columns
data = data.drop(labels=["StockCode", "InvoiceDate", "Country", "CustomerID"], axis=1)
data.head()

,InvoiceNo,Description,Quantity,UnitPrice
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55
1,536365,WHITE METAL LANTERN,6,3.39
2,536365,CREAM CUPID HEARTS COAT HANGER,8,2.75
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,3.39


In [163]:
data.describe()

,Quantity,UnitPrice
count,541909.000000,541909.000000
mean,9.552250,4.611114
std,218.081158,96.759853
min,-80995.000000,-11062.060000
25%,1.000000,1.250000
50%,3.000000,2.080000
75%,10.000000,4.130000
max,80995.000000,38970.000000


In [164]:
neg_unit_prices = data["UnitPrice"] < 0
data[neg_unit_prices]

,InvoiceNo,Description,Quantity,UnitPrice
299983,A563186,Adjust bad debt,1,-11062.06
299984,A563187,Adjust bad debt,1,-11062.06


In [165]:
neg_quantities = data["Quantity"] < 0
len(data[neg_quantities])

10624

In [166]:
data_mod = data[-neg_quantities]
neg_unit_prices = data_mod["UnitPrice"] < 0
data_mod = data_mod[-neg_unit_prices]
len(data_mod)

531283

In [167]:
# threw out
len(data) - len(data_mod)

10626

In [168]:
# what about trash data in Description?
len(data_mod["Description"].unique())

4078

In [169]:
descs = list(data_mod["Description"].unique())
descs = [str(i) for i in descs]
descs = sorted(descs, reverse=True)
descs

['wrongly sold (22719) barcode',
 'wrongly marked 23343',
 'wrongly marked',
 'wrongly coded 23343',
 'wrongly coded 20713',
 'website fixed',
 'to push order througha s stock was ',
 'test',
 'taig adjust',
 'returned',
 'rcvd be air temp fix for dotcom sit',
 'on cargo order',
 'nan',
 'michel oops',
 'mailout ',
 'mailout',
 'incorrectly credited C550456 see 47',
 'had been put aside',
 'found box',
 'found',
 'for online retail orders',
 'dotcomstock',
 'dotcom adjust',
 'dotcom',
 'did  a credit  and did not tick ret',
 'damaged',
 'check?',
 'check',
 'came coded as 20713',
 'amazon sales',
 'amazon adjust',
 'amazon',
 'allocate stock for dotcom orders ta',
 'alan hodge cant mamage this section',
 'adjustment',
 'add stock to allocate online orders',
 'ZINC WIRE SWEETHEART LETTER TRAY',
 'ZINC WIRE KITCHEN ORGANISER',
 'ZINC WILLIE WINKIE  CANDLE STICK',
 'ZINC TOP  2 DOOR WOODEN SHELF ',
 'ZINC T-LIGHT HOLDER STARS SMALL',
 'ZINC T-LIGHT HOLDER STARS LARGE',
 'ZINC T-LIGHT HOLD

In [170]:
# Looks like all-lowercase descriptions are not items
bad_list = descs[0:36]
bad_list.append("Manual")
bad_list.append("nan")
bad_list.append("?")
bad_desc = data_mod["Description"].isin(bad_list)
data_mod = data_mod[-bad_desc]
sum(bad_desc)

463

In [171]:
# how many did we remove total?
len(data) - len(data_mod)

11089

In [172]:
# preprocess description tag
data_mod["Description"] = data_mod["Description"].astype(str)
data_mod["Description"] = data_mod["Description"].apply(str.strip)
data_mod["Description"] = data_mod["Description"].apply(str.lower)
data_mod["Description"] = data_mod["Description"].apply(str.replace, args=('\n',''))
data_mod["Description"] = data_mod["Description"].apply(str.replace, args=('"',''))
data_mod["Description"] = data_mod["Description"].apply(str.replace, args=("'",''))
data_mod["Description"] = data_mod["Description"].apply(str.replace, args=('.',''))
data_mod["Description"] = data_mod["Description"].apply(str.replace, args=('-',''))
data_mod = data_mod.drop(labels=["Quantity", "UnitPrice"], axis=1)
data_mod.head()

,InvoiceNo,Description
0,536365,white hanging heart tlight holder
1,536365,white metal lantern
2,536365,cream cupid hearts coat hanger
3,536365,knitted union flag hot water bottle
4,536365,red woolly hottie white heart


In [151]:
group = data_mod.groupby("InvoiceNo").agg(";".join)
group.head()

,Description
InvoiceNo,
536365,white hanging heart tlight holder;white metal ...
536366,hand warmer union jack;hand warmer red polka dot
536367,assorted colour bird ornament;poppys playhouse...
536368,jam making set with jars;red coat rack paris f...
536369,bath building block word


In [173]:
singles = []
with open("dataset_mod.txt", "w") as f:
    for d in list(group["Description"]):
        if d is not None and d != "":
            f.write("%s\n" % d)
        if len(d.split(";")) == 1:
            singles.append(d)

In [174]:
singles = pd.Series(singles)
singles = singles.unique()
for d in singles:
    print(d)

bath building block word
paper chain kit 50s christmas
victorian sewing box large
jam making set printed
retrospot lamp
homemade jam scented candles
set/5 red retrospot lid glass bowls
nan
washroom metal sign
cream sweetheart egg holder
red  harmonica in box
faux fur chocolate throw
assorted bottle top  magnets
jumbo bag red retrospot
retrospot childrens apron
red woolly hottie white heart
ceramic cake stand + hanging cakes
black record cover frame
3 traditional biscuit cutters  set
green christmas tree card holder
heart of wicker small
lunch box i love london
bank charges
hand warmer owl design
clothes pegs retrospot pack 24
grey heart hot water bottle
assorted colour tlight holder
camouflage led torch
weekend bag vintage rose paisley
white hanging heart tlight holder
swiss roll towel, chocolate  spots
vintage red kitchen cabinet
ocean scent candle jewelled drawer
wrap christmas village
christmas lights 10 vintage baubles
colour glass star tlight holder
regency cakestand 3 tier
charli